In [ ]:
# 验证fastbev pkl文件
# 通过验证，发现fastbev pkl文件中标注信息是在雷达坐标系下

from nuscenes.nuscenes import NuScenes

# 创建NuScenes实例
nusc = NuScenes(version='v1.0-trainval', dataroot='../data/nuscenes', verbose=True)

In [18]:
sample = nusc.sample[0]

sd_rec = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
# lidar to ego
cs_record = nusc.get('calibrated_sensor', sd_rec['calibrated_sensor_token'])
# ego to global
pose_record = nusc.get('ego_pose', sd_rec['ego_pose_token'])

In [19]:
import numpy as np
from pyquaternion import Quaternion

# 获取样本0
ann_token = sample['anns'][0]

# global坐标系
box = nusc.get_box(ann_token)
# global to ego
box.translate(-np.array(pose_record['translation']))
box.rotate(Quaternion(pose_record['rotation']).inverse)
# ego to lidar
box.translate(-np.array(cs_record['translation']))
box.rotate(Quaternion(cs_record['rotation']).inverse)

box

label: nan, score: nan, xyz: [-16.18, -1.17, -1.24], wlh: [0.30, 0.29, 0.73], rot axis: [0.00, 0.00, -1.00], ang(degrees): 168.18, ang(rad): 2.94, vel: nan, nan, nan, name: movable_object.trafficcone, token: 173a50411564442ab195e132472fde71

In [22]:
import mmcv
mmcv.load('../data/nuscenes/pkl/nuscenes_infos_train_4d_interval3_max60.pkl')['infos'][0]['gt_boxes']

array([[-1.61843454e+01, -1.17404151e+00, -1.24046699e+00,
         3.00000000e-01,  2.91000000e-01,  7.34000000e-01,
         1.36454530e+00],
       [-1.54493912e+01, -4.28768163e+00, -1.30136452e+00,
         3.15000000e-01,  3.38000000e-01,  7.12000000e-01,
         1.25993027e+00],
       [-1.02275670e+01,  1.94608211e+01,  3.74364245e-02,
         2.31200000e+00,  7.51600000e+00,  3.09300000e+00,
        -9.64160250e-01],
       [ 9.21442005e+00, -5.57960735e+00, -1.07856950e+00,
         1.63800000e+00,  4.25000000e+00,  1.44000000e+00,
        -1.92929242e+00],
       [-1.57271212e+01, -8.16090985e-01, -6.97936424e-01,
         7.39000000e-01,  5.63000000e-01,  1.71100000e+00,
         1.36454530e+00],
       [ 3.84646471e-01, -1.32284491e+01, -1.21462740e+00,
         1.87100000e+00,  4.47800000e+00,  1.45600000e+00,
        -2.66400622e+00],
       [-4.75276596e+01,  3.51366615e+01,  6.94957388e-01,
         2.87700000e+00,  6.37200000e+00,  2.97800000e+00,
        -4.1599453

In [5]:
# 验证fastbev与sabev的标签下标是否一致 - 不一致
import mmcv
# fast bev 样本
f_sample_0 = mmcv.load('../data/nuscenes/pkl/nuscenes_infos_train_4d_interval3_max60.pkl')['infos'][0]
# bevdet 样本
b_sample_0 = mmcv.load('../data/nuscenes/pkl/bevdetv2-nuscenes_infos_train.pkl')['infos'][0]

In [6]:
# ['traffic_cone', 'traffic_cone', 'truck', 'car', 'pedestrian', 'car', 'truck', 'pedestrian', 'traffic_cone', 'truck']
# [9, 9, 1, 0, 8, 0, 1, 8, 9, 1]
f_sample_0, b_sample_0

({'lidar_path': './data/nuscenes/samples/LIDAR_TOP/n015-2018-07-18-11-07-57+0800__LIDAR_TOP__1531883530449377.pcd.bin',
  'token': 'e93e98b63d3b40209056d129dc53ceee',
  'sweeps': [],
  'cams': {'CAM_FRONT': {'data_path': './data/nuscenes/samples/CAM_FRONT/n015-2018-07-18-11-07-57+0800__CAM_FRONT__1531883530412470.jpg',
    'type': 'CAM_FRONT',
    'sample_data_token': '020d7b4f858147558106c504f7f31bef',
    'sensor2ego_translation': [1.70079118954, 0.0159456324149, 1.51095763913],
    'sensor2ego_rotation': [0.4998015430569128,
     -0.5030316162024876,
     0.4997798114386805,
     -0.49737083824542755],
    'ego2global_translation': [1010.1102882349232, 610.6567106479714, 0.0],
    'ego2global_rotation': [-0.7530285141171715,
     -0.007718682910458633,
     0.00863090844122062,
     -0.6578859979358822],
    'timestamp': 1531883530412470,
    'sensor2lidar_rotation': array([[ 0.99995012,  0.00730543,  0.00681137],
           [-0.00694924,  0.01901527,  0.99979504],
           [ 0.00